<a href="https://colab.research.google.com/github/scancer-org/data-prep-and-model-train/blob/main/Copy_of_PCAM_Code_%2B_Documentation_Data%2C_Model%2C_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [PCAM Classification](https://github.com/basveeling/pcam) Project
## FSDL Online Course - Spring 2021
## Daniel Hen, Harish Narayanan

In [1]:
### TODO:
# 1. Test on test dataloader (final results)
# 2. Insert transformations to data - done, just validate by printing and showing some images
# 4. Study how to save after training in torch-serve-archiver file (.mar file)
# 5. Clean + Document code and have a train for at least 50 epochs

### Installing Required Packages

In [2]:
!pip install -qqq wandb

     |████████████████████████████████| 2.1MB 2.9MB/s 
     |████████████████████████████████| 102kB 8.3MB/s 
     |████████████████████████████████| 133kB 17.7MB/s 
     |████████████████████████████████| 163kB 18.4MB/s 
     |████████████████████████████████| 71kB 8.1MB/s 


### Libraries + Functions import

In [3]:
# Test labels (from original, which I'll split for train / test)
import h5py
import numpy as np
import torch
import wandb
import os
import pandas as pd
import PIL.Image
import matplotlib.pyplot as plt
import shutil
import time
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from google.colab import drive
from torch.utils import data
from os import listdir
from pathlib import Path
from PIL import Image
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, datasets

### Weights & Biases parameters

In [4]:
wandb.login()
wandb.init(project="pcam-pytorch-training")
wandb.run.name = "pcam-pytorch-experiment#-" + wandb.run.id
print("Staring experiment: ", wandb.run.name)

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hnarayanan (use `wandb login --relogin` to force relogin)


Staring experiment:  pcam-pytorch-experiment#-3migfszg


### Google Drive Mounting - for being able to easily read the data

In [5]:
drive.mount('/content/gdrive/')
!ls gdrive/MyDrive/pcamv1

Mounted at /content/gdrive/
camelyonpatch_level_2_split_test_meta.csv
camelyonpatch_level_2_split_test_x.h5
camelyonpatch_level_2_split_test_y.h5
camelyonpatch_level_2_split_train_mask.h5
camelyonpatch_level_2_split_train_meta.csv
camelyonpatch_level_2_split_train_x.h5
camelyonpatch_level_2_split_train_y.h5
camelyonpatch_level_2_split_valid_meta.csv
camelyonpatch_level_2_split_valid_x.h5
camelyonpatch_level_2_split_valid_y.h5


### Class H5Dataset:
Defines our dataset class in which we will load data from.
<br>
Also, deals with hdfs file format, which requires a customized reference in PyTorch

In [6]:
class H5Dataset(Dataset):
    def __init__(self, path, transform=None):
        self.file_path = path
        self.dataset_x = None
        self.dataset_y = None
        self.transform = transform
        ### Going to read the X part of the dataset - it's a different file
        with h5py.File(self.file_path + '_x.h5', 'r') as filex:
            self.dataset_x_len = len(filex['x'])

        ### Going to read the y part of the dataset - it's a different file
        with h5py.File(self.file_path + '_y.h5', 'r') as filey:
            self.dataset_y_len = len(filey['y'])

    def __len__(self):
        assert self.dataset_x_len == self.dataset_y_len # Since we are reading from different sources, validating we are good in terms of size both X, Y
        return self.dataset_x_len

    def __getitem__(self, index):
        imgs_path = self.file_path + '_x.h5'
        labels_path = self.file_path + '_y.h5'

        if self.dataset_x is None:
            self.dataset_x = h5py.File(imgs_path, 'r')['x']
        if self.dataset_y is None:
            self.dataset_y = h5py.File(labels_path, 'r')['y']

        # get one pair of X, Y and return them, transform if needed
        image = self.dataset_x[index]
        label = self.dataset_y[index]

        if self.transform:
            image = self.transform(image)

        return (image, label)

### Configurations params, Dataset + Dataloader Instantiation

In [7]:
CHECKPOINT_DIR = 'checkpoint'
drive_base_path = 'gdrive/MyDrive/pcamv1/'
BATCH_SIZE = 16
dataloader_params = {'batch_size': BATCH_SIZE, 'shuffle': True, 'num_workers': 2}

train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_path = drive_base_path + 'camelyonpatch_level_2_split_train'
val_path = drive_base_path + 'camelyonpatch_level_2_split_valid'
test_path = drive_base_path + 'camelyonpatch_level_2_split_test'

test_dataset = H5Dataset(test_path, transform=test_transforms)
test_loader = DataLoader(test_dataset, **dataloader_params)

val_dataset = H5Dataset(val_path, transform=test_transforms)
dev_loader = DataLoader(val_dataset, **dataloader_params)

train_dataset = H5Dataset(train_path, transform=train_transforms)
train_loader = DataLoader(train_dataset, **dataloader_params)

### Model Architecture Class Definition

In [8]:
class CNN_V2(nn.Module):
    """Implemented by paper: http://cs230.stanford.edu/projects_winter_2019/posters/15813053.pdf"""
    def __init__(self, p = 0.5):
        # log dropout parameter
        wandb.config.dropout = p
        """Init method for initializaing the CNN model"""
        super(CNN_V2, self).__init__()
        # 1. Convolutional layers
        # Single image is in shape: 3x96x96 (CxHxW, H==W), RGB images
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 3, stride = 1, padding = 1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        
        self.dropout = nn.Dropout(p = p)
        
        # 2. FC layers to final output
        self.fc1 = nn.Linear(in_features = 128*6*6, out_features = 512)
        self.fc_bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(in_features = 512, out_features = 256)
        self.fc_bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(in_features = 256, out_features = 128)
        self.fc_bn3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(in_features = 128, out_features = 1)

    def forward(self, x):
        # Convolution Layers, followed by Batch Normalizations, Maxpool, and ReLU
        x = self.bn1(self.conv1(x))                      # batch_size x 96 x 96 x 16
        x = self.pool(F.relu(x))                         # batch_size x 48 x 48 x 16
        x = self.bn2(self.conv2(x))                      # batch_size x 48 x 48 x 32
        x = self.pool(F.relu(x))                         # batch_size x 24 x 24 x 32
        x = self.bn3(self.conv3(x))                      # batch_size x 24 x 24 x 64
        x = self.pool(F.relu(x))                         # batch_size x 12 x 12 x 64
        x = self.bn4(self.conv4(x))                      # batch_size x 12 x 12 x 128
        x = self.pool(F.relu(x))                         # batch_size x  6 x  6 x 128
        # Flatten the output for each image
        x = x.reshape(-1, self.num_flat_features(x))        # batch_size x 6*6*128
        
        # Apply 4 FC Layers
        x = self.fc1(x)
        x = self.fc_bn1(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        x = self.fc_bn2(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = self.fc3(x)
        x = self.fc_bn3(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = self.fc4(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

### Helper Functions

In [9]:
def sigmoid(x):
    """This method calculates the sigmoid function"""
    return 1.0/(1.0 + np.exp(-x))

def training_accuracy(predicted, true, i, acc, tpr, tnr):
    """Taken from https://www.kaggle.com/krishanudb/cancer-detection-deep-learning-model-using-pytorch"""
    predicted = predicted.cpu() # Taking the predictions, why cpu and not device?
    true = true.cpu() # Taking the labels, why cpu and not device?
    
    predicted = (sigmoid(predicted.data.numpy()) > 0.5) # Using sigmoid above, if prediction > 0.5 it is 1
    true = true.data.numpy() # Numpy - can't combine above?
    accuracy = np.sum(predicted == true) / true.shape[0] # Accuracy is: (TP + TN)/(TP + TN + FN + FP)
    true_positive_rate = np.sum((predicted == 1) * (true == 1)) / np.sum(true == 1) # TPR: TP / (TP + FN) aka Recall
    true_negative_rate = np.sum((predicted == 0) * (true == 0)) / np.sum(true == 0) # TNR: TN / (FP + TN)
    acc = acc * (i) / (i + 1) + accuracy / (i + 1)
    tpr = tpr * (i) / (i + 1) + true_positive_rate / (i + 1)
    tnr = tnr * (i) / (i + 1) + true_negative_rate / (i + 1)
    return acc, tpr, tnr

def dev_accuracy(predicted, target):
    """Taken from https://www.kaggle.com/krishanudb/cancer-detection-deep-learning-model-using-pytorch"""
    predicted = predicted.cpu()
    target = target.cpu()
    predicted = (sigmoid(predicted.data.numpy()) > 0.5)
    true = target.data.numpy()
    accuracy = np.sum(predicted == true) / true.shape[0]
    true_positive_rate = np.sum((predicted == 1) * (true == 1)) / np.sum(true == 1)
    true_negative_rate = np.sum((predicted == 0) * (true == 0)) / np.sum(true == 0)
    return accuracy, true_positive_rate, true_negative_rate

# def train_dev_test_split_indices(dataset_size, train_split=0.8, dev_split=0.1, seed=30):
#     """Given a dataset size, returns the train/dev/test split indices"""
#     indices = list(range(dataset_size))
#     train_split_end = int(np.floor(train_split * dataset_size))
#     dev_split_end = int(np.floor(dev_split * dataset_size)) + train_split_end
#     np.random.seed(seed)
#     np.random.shuffle(indices)
#     train_indices = indices[:train_split_end]
#     dev_indices = indices[train_split_end:dev_split_end]
#     test_indices = indices[dev_split_end:]
#     return train_indices, dev_indices, test_indices

# def train_dev_test_split_indices_from_dataset(dataset, train_split=0.8, dev_split=0.1, seed=30):
#     """Given a dataset, returns the train/dev/test split indices"""
#     dataset_size = len(dataset)
#     indices = list(range(dataset_size))
#     train_split_end = int(np.floor(train_split * dataset_size))
#     dev_split_end = int(np.floor(dev_split * dataset_size)) + train_split_end
#     np.random.seed(seed)
#     np.random.shuffle(indices)
#     train_indices = indices[:train_split_end]
#     dev_indices = indices[train_split_end:dev_split_end]
#     test_indices = indices[dev_split_end:]
#     return train_indices, dev_indices, test_indices

def fetch_state(epoch, model, optimizer, dev_loss_min, dev_acc_max):
    """Returns the state dictionary for a model and optimizer"""
    state = {
        'epoch': epoch,
        'dev_loss_min': dev_loss_min,
        'dev_acc_max': dev_acc_max,
        'state_dict': model.state_dict(),
        'optim_dict': optimizer.state_dict()
    }
    return state

def save_checkpoint(state, is_best = False, checkpoint = CHECKPOINT_DIR):
    """Taken from CS230 PyTorch Code Examples"""
    """Saves model and training parameters at checkpoint + 'last.pth.tar'. If is_best==True, also saves
    checkpoint + 'best.pth.tar'

    Args:
        state: (dict) contains model's state_dict, may contain other keys such as epoch, optimizer state_dict
        is_best: (bool) True if it is the best model seen till now
        checkpoint: (string) folder where parameters are to be saved
    """
    filepath = os.path.join(checkpoint, 'last_v2.pth.tar')
    if (not os.path.exists(checkpoint)):
        print("Checkpoint Directory does not exist! Making directory {}".format(checkpoint))
        os.mkdir(checkpoint)
    else:
        print("Checkpoint Directory exists! ")
    torch.save(state, filepath)
    if (is_best):
        shutil.copyfile(filepath, os.path.join(checkpoint, 'best_v2.pth.tar'))
        
def load_checkpoint(model, optimizer = None, checkpoint = CHECKPOINT_DIR):
    """Taken from CS230 PyTorch Code Examples"""
    """Loads model parameters (state_dict) from file_path. If optimizer is provided, loads state_dict of
    optimizer assuming it is present in checkpoint.

    Args:
        checkpoint: (string) filename which needs to be loaded
        model: (torch.nn.Module) model for which the parameters are loaded
        optimizer: (torch.optim) optional: resume optimizer from checkpoint
    """
    if not os.path.exists(checkpoint):
        print("File doesn't exist {}".format(checkpoint))
        checkpoint = None
        return
    checkpoint = torch.load(checkpoint)
    model.load_state_dict(checkpoint['state_dict'])

    if optimizer:
        optimizer.load_state_dict(checkpoint['optim_dict'])

    return checkpoint

In [10]:
# Load model
USE_GPU = torch.cuda.is_available()

model = CNN_V2()
if (USE_GPU):
    model.cuda()

# Hyperparameters
lr = 5e-4
wandb.config.learning_rate = lr

# Parameters
num_workers = 0
total_epochs = 0
num_epochs = 1
early_stop_limit = 10
bad_epoch_count = 0
stop = False
train_loss_min = np.Inf
dev_loss_min = np.Inf
dev_acc_max = 0

In [11]:
# Optimizer + Loss Function
#optimizer = optim.Adam(model.parameters(), lr = lr)
optimizer = optim.SGD(model.parameters(), lr = lr)   # SWATS
criterion = nn.BCEWithLogitsLoss() # Binary Cross Entropy for binary classification - malignant / benign

# Load best checkpoint
best_checkpoint = os.path.join(CHECKPOINT_DIR, 'best_v2.pth.tar');
#checkpoint = load_checkpoint(model = model, optimizer = optimizer, checkpoint = best_checkpoint)
# checkpoint = load_checkpoint(model = model, optimizer = None, checkpoint = best_checkpoint) # SWATS
# total_epochs = None if checkpoint is None else checkpoint['epoch']
total_epochs = 0

# Initialize arrays for plot
train_loss_arr = []
train_acc_arr = []
train_tpr_arr = []
train_tnr_arr = []

dev_loss_arr = []
dev_acc_arr = []
dev_tpr_arr = []
dev_tnr_arr = []

In [ ]:
# Loop over the dataset multiple times
total_num_epochs = total_epochs + num_epochs
for epoch in range(num_epochs):
    curr_epoch = total_epochs + epoch + 1
    # Keep track of training loss
    train_loss = []
    # Keep track of dev loss
    dev_loss = []
    # Keep track of accuracy measurements
    acc, tpr, tnr = 0.0, 0.0, 0.0

    # Train the model
    start_time = time.time()
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        if USE_GPU:
            data, target = image.cuda(), label.cuda()
        else:
            data, target = image, label
        # Zero the parameter gradients
        optimizer.zero_grad()
        # Forward pass
        output = model(data)
        # Update target to be the same dimensions as output
        target = target.view(output.shape[0], 1).float()
        # Get accuracy measurements
        acc, tpr, tnr = training_accuracy(output, target, batch_idx, acc, tpr, tnr)
        # Calculate the batch's loss
        curr_train_loss = criterion(output, target)
        # Update the training loss
        train_loss.append(curr_train_loss.item())
        # Backward pass
        curr_train_loss.backward()
        # Perform a single optimization step to update parameters
        optimizer.step()
        # Print debug info every 64 batches
        if (batch_idx) % 64 == 0:
            print('Epoch {}/{}; Iter {}/{}; Loss: {:.4f}; Acc: {:.3f}; True Pos: {:.3f}; True Neg: {:.3f}'
                   .format(curr_epoch, total_num_epochs, batch_idx + 1, len(train_loader), curr_train_loss.item(), acc, tpr, tnr))
            
    end_time = time.time()
    
    # Evaluate the model
    model.eval()
    with torch.no_grad():
        for batch_idx, (image, label) in enumerate(dev_loader):
            if USE_GPU:
                data, target = image.cuda(), label.cuda()
            else:
                data, target = image, label
            # Get predicted output
            output = model(data)
            # Update target to be the same dimensions as output
            target = target.view(output.shape[0], 1).float()
            # Get accuracy measurements
            dev_acc, dev_tpr, dev_tnr = dev_accuracy(output, target)
            # Calculate the batch's loss
            curr_dev_loss = criterion(output, target)
            # Update the dev loss
            dev_loss.append(curr_dev_loss.item())
    
    # Calculate average loss
    avg_train_loss = np.mean(np.array(train_loss))
    avg_dev_loss = np.mean(np.array(dev_loss))
    
    # Update dev loss arrays
    dev_loss_arr.append(avg_dev_loss)
    dev_acc_arr.append(dev_acc)
    dev_tpr_arr.append(dev_tpr)
    dev_tnr_arr.append(dev_tnr)

    # Update training loss arrays
    train_loss_arr.append(avg_train_loss)
    train_acc_arr.append(acc)
    train_tpr_arr.append(tpr)
    train_tnr_arr.append(tnr)

    print('Epoch {}/{}; Avg. Train Loss: {:.4f}; Train Acc: {:.3f}; Train TPR: {:.3f}; Train TNR: {:.3f}; Epoch Time: {} mins; \nAvg. Dev Loss: {:.4f}; Dev Acc: {:.3f}; Dev TPR: {:.3f}; Dev TNR: {:.3f}\n'
        .format(curr_epoch, total_num_epochs, avg_train_loss, acc, tpr, tnr, round((end_time - start_time)/ 60., 2), avg_dev_loss, dev_acc, dev_tpr, dev_tnr))
    
    wandb.log({'epoch': curr_epoch, 'loss': avg_train_loss, 'accuracy': acc, 'tpr': tpr, 'time_per_epoch_min': round((end_time - start_time)/ 60., 2)})

    if avg_dev_loss < dev_loss_min:
        print('Dev loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
              .format(dev_loss_min, avg_dev_loss))
        dev_loss_min = avg_dev_loss
        is_best = False
        if (dev_acc >= dev_acc_max):
            is_best = True
            dev_acc_max = dev_acc
        state = fetch_state(epoch = curr_epoch, model = model, optimizer = optimizer, 
                            dev_loss_min = dev_loss_min, 
                            dev_acc_max = dev_acc_max)
        save_checkpoint(state = state, is_best = is_best)
        bad_epoch_count = 0
    # If dev loss didn't improve, increase bad_epoch_count and stop if
    # bad_epoch_count >= early_stop_limit
    else:
        bad_epoch_count += 1
        print('{} epochs of increasing dev loss ({:.6f} --> {:.6f}).'
              .format(bad_epoch_count, dev_loss_min, avg_dev_loss))
        if (bad_epoch_count >= early_stop_limit):
            print('Stopping training')
            stop = True

    if (stop):
        break

Epoch 1/1; Iter 1/16384; Loss: 0.7109; Acc: 0.562; True Pos: 0.833; True Neg: 0.400
Epoch 1/1; Iter 65/16384; Loss: 0.5951; Acc: 0.498; True Pos: 0.543; True Neg: 0.459
Epoch 1/1; Iter 129/16384; Loss: 0.6088; Acc: 0.507; True Pos: 0.534; True Neg: 0.482
Epoch 1/1; Iter 193/16384; Loss: 0.6760; Acc: 0.528; True Pos: 0.547; True Neg: 0.512
Epoch 1/1; Iter 257/16384; Loss: 0.7358; Acc: 0.540; True Pos: 0.552; True Neg: 0.537
Epoch 1/1; Iter 321/16384; Loss: 0.6605; Acc: 0.554; True Pos: 0.561; True Neg: 0.557
Epoch 1/1; Iter 385/16384; Loss: 0.6456; Acc: 0.569; True Pos: 0.575; True Neg: 0.572
Epoch 1/1; Iter 449/16384; Loss: 0.5869; Acc: 0.585; True Pos: 0.589; True Neg: 0.591
Epoch 1/1; Iter 513/16384; Loss: 0.5843; Acc: 0.595; True Pos: 0.599; True Neg: 0.603
Epoch 1/1; Iter 577/16384; Loss: 0.5941; Acc: 0.604; True Pos: 0.606; True Neg: 0.615
Epoch 1/1; Iter 641/16384; Loss: 0.4511; Acc: 0.615; True Pos: 0.617; True Neg: 0.628
Epoch 1/1; Iter 705/16384; Loss: 0.5430; Acc: 0.621; True

In [ ]:
model.state_dict

In [ ]:
model

In [ ]:
help(torch.save)

In [ ]:
# Possibly important, we need to switch the model from
# training mode to evaluation mode

model.eval()

In [ ]:
# Option 1: The basic one, as a Python pickle. In the TorchServe
# documentation, they call this "eager mode" models

torch.save(model, 'CNNv2_1.pt')

In [ ]:
# Option 2: What the TorchServe documentation refers to as
# "scripted mode" models

scripted = torch.jit.script(model)
scripted.save("CNNv2_2.pt")

In [ ]:
# Option 3: Traced mode, not even referred to in the TorchServe
# documentation

example_input = torch.rand(1, 3, 96, 96).to('cuda:0')
traced_script = torch.jit.trace(model, example_input)
traced_script.save("CNNv2_3.pt")

In [ ]:
# Option 4: Eager mode that saves the "state dictionary"
# representaton of the model

state_dict = model.state_dict()
torch.save(state_dict, 'CNNv2_4.pt')